In [1]:
import numpy as np
import pandas as pd
import math
import requests 
import matplotlib as plt
import seaborn as sns
import yfinance as yf
import pandas_datareader as web
from pandas_datareader import data
from bs4 import BeautifulSoup as bs
from scipy import stats

In [2]:
tick = pd.read_csv("all_stocks")

In [32]:
final = pd.DataFrame(columns=['Ticker','Price','Trailing PE ratio', 'Forward PE ratio','Shares to buy'])
for i in range(len(tick["0"])):
    print("Producing data for: ", tick["0"][i])
    try:
        ticker = yf.Ticker(tick["0"][i])
        info = ticker.financials
        print(info)
        #for i in info: 
        #    if "amorti" in i.lower(): 
        #        print(i)
        #In info: 
            #We have price to book
            #We have price to sale 12 months
            #We have enterprise value
            #We have gross profit
        #In financials: 
        #TODO

        #Ratio endettement (liabilities/assets)
        #Working capital (currentAssets/currentLiability) Voir si depense sont capapble d'etre couverte dans la prpchaine annee
    except TypeError: 
        print("Nonetype found for: " + tick["0"][i])
        continue
    except IndexError: 
        print("Couldn't find: ",tick["0"][i])
        continue
    except KeyError:
        print("Couldnt find key for: " + tick['0'][i])
        continue
    break

    '''    
    try: 
        pass
    except IndexError: 
        continue
    #Change to keep only last 2 days!!! get date
    
    #Append latest value
    final = final.append(
        pd.Series(d,
        index=['Ticker','Price','Trailing PE ratio', 'Forward PE ratio','Shares to buy']), 
        ignore_index = True
    ) 
    '''

Producing data for:  AOS
                                                      2021-12-31  \
Total Revenue                                       3538900000.0   
Operating Revenue                                   3538900000.0   
Cost Of Revenue                                     2228000000.0   
Gross Profit                                        1310900000.0   
Operating Expense                                    701400000.0   
Selling General And Administration                   701400000.0   
Operating Income                                     609500000.0   
Net Non Operating Interest Income Expense             -4300000.0   
Interest Expense Non Operating                         4300000.0   
Other Income Expense                                  20400000.0   
Special Income Charges                                       0.0   
Restructuring And Mergern Acquisition                        0.0   
Other Non Operating Income Expenses                   20400000.0   
Pretax Income          

In [5]:
final.to_csv("value_invest_advanced_sheet")